In [2]:

import pandas as pd

from pathlib import Path 
current_path = Path().resolve()

RAW_PATH = current_path.parent / 'data' / 'raw'
OUTPUT_PATH = current_path.parent / 'data' / 'output'

pd.set_option('display.max_columns', 500)

%matplotlib inline

from h3 import h3
from copy import deepcopy

ModuleNotFoundError: No module named 'h3'

In [323]:
df = pd.read_csv(RAW_PATH / 'h3_address.csv', names=['h3_address'])

In [328]:
def to_geojson(df, h3_address='h3_address'):
    
    swipe = lambda x: [x[1], x[0]]

    df['h3_bound'] = df[h3_address].apply(h3.h3_to_geo_boundary)
    df['h3_bound'] = df['h3_bound'].apply(lambda x: list(map(swipe, x)))
    df['h3_bound'] = df['h3_bound'].apply(lambda x: [x + [x[0]]])
    
    geojson = {'features': [],
           'type': 'FeatureCollection'}
    
    feature = {'geometry': 
           {'coordinates': None,
            'type': 'Polygon'},
           'properties': {'id': None},
           'type': 'Feature'}
    
    for i, row in df.iterrows():
    
        _feature = deepcopy(feature)

        _feature['geometry']['coordinates'] = row['h3_bound']
        _feature['properties'] = {'h3_address': row['h3_address']}

        geojson['features'].append(_feature)
    
    return geojson

In [330]:
geojson = to_geojson(df)

In [332]:
json.dump(geojson, open(RAW_PATH / 'h3_address.geojson', 'w'))